<a href="https://colab.research.google.com/github/Samdersson/Ecuaciones-no-lineales/blob/main/ecuaciones_no_lineales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# metodo de Newto-Rpahson

Ejercicio 1

In [12]:
import numpy as np

# Definimos la función de error como una función no lineal de los pesos w1, w2 y b
def error(X):
    w1, w2, b = X
    return np.array([
        np.exp(w1) + w2 + b - 1,  # Primera ecuación que involucra los pesos
        w1**2 + w2**2 - 4,  # Segunda ecuación (penaliza valores altos de los pesos)
        w1 + w2 + b - 2  # Tercera ecuación (regularización simple)
    ])

# Jacobiano de la función de error (derivadas parciales con respecto a los pesos)
def jacobiano(X):
    w1, w2, b = X
    return np.array([
        [np.exp(w1), 1, 1],  # Derivada de la primera ecuación respecto a w1, w2 y b
        [2*w1, 2*w2, 0],  # Derivada de la segunda ecuación respecto a w1, w2 y b
        [1, 1, 1]  # Derivada de la tercera ecuación respecto a w1, w2 y b
    ])

# Método de Newton-Raphson para encontrar los pesos óptimos
def newton_raphson(error, J, X0, tol=1e-6, max_iter=100):
    X = X0
    for i in range(max_iter):
        J_inv = np.linalg.inv(J(X))  # Invertir el jacobiano en el punto actual
        F_val = error(X)  # Evaluar la función de error en el punto actual
        X_new = X - np.dot(J_inv, F_val)  # Actualizar los pesos
        if np.linalg.norm(X_new - X, ord=np.inf) < tol:  # Verificar convergencia
            return X_new
        X = X_new  # Actualizar los pesos actuales
    print (" El método no converge después de {} iteraciones".format(max_iter))
    return X

# Estimación inicial de los pesos y bias
X0 = np.array([0.5, 0.5, 0.5])
# Llamada al método de Newton-Raphson para minimizar el error
solucion = newton_raphson(error, jacobiano, X0)
print("Pesos y bias óptimos:", solucion)


 El método no converge después de 100 iteraciones
Pesos y bias óptimos: [16.50321348 -8.42153708 -6.0816764 ]


# 2 Método de Broyden

In [2]:
import numpy as np
from scipy.optimize import broyden1

# Función de error no lineal
def error(X):
    w1, w2, b = X
    return np.array([
        w1**2 + w2**2 + b**2 - 5,  # Función de error sobre los pesos
        np.tanh(w1) + w2 + b - 1,  # Función de activación no lineal
        w1 + w2 + b - 2  # Regularización simple
    ])

# Método de Broyden sin calcular derivadas explícitamente
X0 = np.array([1.0, 1.0, 1.0])  # Estimación inicial de los pesos
solucion = broyden1(error, X0, f_tol=1e-6)  # Uso del método de Broyden
print("Pesos óptimos:", solucion)


Pesos óptimos: [ 1.9611792  -0.7398733   0.77869377]


# 3. Método de Punto Fijo

In [8]:
import numpy as np

# Definimos la función de ajuste para el método de punto fijo
def G(X):
    w1, w2, b = X
    return np.array([
        np.sqrt(4 - w2**2),  # Aproximación para w1
        1 - np.exp(w1),  # Aproximación para w2
        w1 + w2 - 2  # Aproximación para b
    ])

# Método de Punto Fijo para ajustar los pesos
def punto_fijo(G, X0, tol=1e-6, max_iter=100):
    X = X0
    for i in range(max_iter):
        X_new = G(X)  # Aplicar la función G para actualizar los pesos
        if np.linalg.norm(X_new - X, ord=np.inf) < tol:  # Verificar convergencia
            return X_new
        X = X_new  # Actualizar los pesos actuales
    print ("warnning: El método no converge después de {} iteraciones".format(max_iter))
    return X

# Estimación inicial de los pesos
X0 = np.array([1.0, 1.0, 1.0])
# Llamada al método de punto fijo para ajustar los pesos
solucion = punto_fijo(G, X0)
print("Pesos óptimos:", solucion)


warnning: El método no converge después de 100 iteraciones
Pesos óptimos: [nan nan nan]


<ipython-input-8-1fc3c9ea3c5f>:7: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(4 - w2**2),  # Aproximación para w1


# 4. Método de la Secante Multivariable

In [11]:
# importamos las librerias necesarias en este caso solo es una
import numpy as np

# Definimos la función de error que depende de los pesos w1, w2 y el bias b
def error(X):
    w1, w2, b = X
    return np.array([
        w1**2 + w2**2 + b**2 - 5,  # Función de error (penaliza grandes valores de los pesos)
        np.tanh(w1) + w2 + b - 1,  # Activación no lineal con tangente hiperbólica
        w1 + w2 + b - 2  # Regularización simple
    ])

# Método de la Secante Multivariable
def secante_multivariable(F, X0, X1, tol=1e-6, max_iter=100):
    for i in range(max_iter):
        F0, F1 = F(X0), F(X1)  # Evaluamos la función de error en X0 y X1
        diff = X1 - X0  # Diferencia entre X1 y X0
        delta_F = F1 - F0  # Diferencia entre F(X1) y F(X0)

        if np.linalg.norm(delta_F, ord=np.inf) < tol:  # Si la diferencia es pequeña, hemos convergido
            return X1

        # Actualización de los parámetros
        X2 = X1 - np.dot(diff, F1) / np.dot(delta_F, delta_F) * delta_F
        X0, X1 = X1, X2  # Actualizamos las estimaciones para la siguiente iteración

    raise ValueError("El método no converge después de {} iteraciones".format(max_iter))

# Estimación inicial de los pesos y bias
X0 = np.array([1.0, 1.0, 1.0])  # Primer punto
X1 = np.array([0.9, 0.9, 0.9])  # Segundo punto cercano al primero

# Aplicamos el método de la secante multivariable
solucion = secante_multivariable(error, X0, X1)
print("Pesos y bias óptimos:", solucion)


Pesos y bias óptimos: [1.28610341 1.04218462 1.09962675]
